In [1]:
import pandas as pd
import numpy as np

In [3]:
sample_size = 2000

In [4]:
channels_df = pd.read_csv('channels_sample_{}.csv'.format(sample_size), encoding='ISO-8859-1')

C:\Users\admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
from sklearn.cluster import KMeans

In [131]:
def preprocess(X):
    """
    Reduces matrix dimension into dimension with most viable features
    Return df contains:
        'followers': int
        'views': int
        # 'follow_ct': int, size of follows list
        # 'len_disp_name': int, len of display name
        # 'disp_name_exoticness': int, zaniness of the username
        # 'lang_idx': int, index associated with the channel language
        # 'broadcaster_lang_idx': int, index associated with the broadcast language
        'game_idx': int, index associated with the game
    """
    t = X.dropna(subset=['views', 'follows', 'broadcaster_language', 'followers', 'language', 'id'], how='any')
    t = t[['views', 'broadcaster_language', 'display_name', 'followers', 'game', 'language', 'id', 'follows']]
    t['followers'] = t['followers'].map(int)
    t['views'] = t['views'].map(int)
#     t['follow_ct'] = t['follows'].map(len)
#     t['len_disp_name'] = t['display_name'].map(len)
#     t['disp_name_exoticness'] = t['display_name'].map(lambda x: sum([2 if i.isupper() or i.isnumeric() else 1 for i in x]) + len(x)-10);
#     t['lang_idx'] = t['language'].map({lang: num for num, lang in enumerate(t['language'].unique())})
#     t['broadcaster_lang_idx'] = t['broadcaster_language'].map({bcl: num for num, bcl in enumerate(t['broadcaster_language'].unique())})
    t['game_idx'] = t['game'].map({game: num for num, game in enumerate(t['game'].unique())})
    print(t.shape)
    print(t.dtypes)
    return t

In [132]:
X = preprocess(channels_df)
X.head()

(1667, 9)
views                    int64
broadcaster_language    object
display_name            object
followers                int64
game                    object
language                object
id                      object
follows                 object
game_idx                 int64
dtype: object


,views,broadcaster_language,display_name,followers,game,language,id,follows,game_idx
0,354301250,en,Ninja,11884880,Fortnite,en,19571641,"['128479231', '46386566', '214062798', '705436...",0
1,226571889,en,shroud,4681624,Call of Duty: Black Ops 4,en,37402112,"['29140917', '6978352', '132230344', '10406', ...",1
2,62085343,en,TSM_Myth,4440552,Fortnite,en,110690086,"['70661496', '186356535', '161719481', '240936...",0
3,245220630,en,summit1g,3320497,Call of Duty: Black Ops 4,en,26490481,"['134433708', '78219897', '26301881', '1668410...",1
4,47709889,en,Tfue,3238391,Fortnite,en,60056333,"['106941811', '145151151', '47229052', '130149...",0


In [159]:
kmeans = KMeans(n_clusters=15, random_state=14).fit(X.select_dtypes(include=['int64']))
X['pred_cluster'] = kmeans.predict(X.select_dtypes(include=['int64']))
X.groupby('pred_cluster').mean()

,views,followers,game_idx
pred_cluster,,,
0,6.516846e+05,3.011767e+04,58.967527
1,2.967428e+08,2.456910e+06,3.000000
2,1.063975e+09,3.229998e+06,2.000000
3,6.405676e+07,8.451147e+05,16.090909
4,2.024812e+08,1.429576e+06,8.500000
5,2.849403e+07,5.506235e+05,19.466667
6,1.544035e+07,3.290609e+05,24.140351
7,5.377027e+08,1.241030e+05,9.000000
8,1.102615e+08,8.497379e+05,23.000000


In [160]:
X.groupby('pred_cluster').views.count()

pred_cluster
0     1201
1        2
2        1
3       22
4        4
5       60
6      114
7        1
8       14
9        3
10       5
11       4
12      40
13       9
14     187
Name: views, dtype: int64

In [161]:
X[X['pred_cluster']==0]

,views,broadcaster_language,display_name,followers,game,language,id,follows,game_idx,pred_cluster
91,3361954,en,MiniLaddd,605882,Call of Duty: Black Ops 4,en,24057992,"['15564828', '78064636', '106125347', '1665718...",1,0
110,446996,en,ORIGINPC,545343,Red Dead Redemption 2,en,56728613,"['96074667', '7236692', '124510977', '4123340'...",9,0
133,3075365,es,Luzu,487250,Just Chatting,es,66370849,"['75802639', '55125740', '197886470', '2218697...",3,0
148,13795,en,ORIGINPCCEO,455546,IRL,en,35934920,"['99292239', '106798225', '4123340', '19571641...",39,0
184,3127347,ar,RakanooLive,378694,Fortnite,en,119638640,"['139807953', '37402112', '82479391', '7759128...",0,0
186,874020,en,PostMalone,369287,PLAYERUNKNOWN'S BATTLEGROUNDS,en,177782786,"['37402112', '46021592', '19264788', '55694098...",31,0
187,2386143,en,FaNg_I3aNGeR,363284,Fortnite,en,25877830,"['17337557', '127506955', '501281', '25163635'...",0,0
215,1413472,en,muselk,330703,Battlefield V,en,38340208,"['31595348', '30381987', '14408894', '64788017...",50,0
230,2290981,fr,LeBouseuh,311624,Fortnite,fr,96562014,"['148043031', '88959886', '73569213', '2764192...",0,0
240,3039258,en,Yelo,304274,Fortnite,en,32526505,"['123282882', '132843382', '67143805', '108421...",0,0
